In [490]:
# Config Data Structure
import pandas as pd
from datetime import datetime as dt
from pymongo import MongoClient as mc
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from collections import Counter
import pprint as pp
import random as ran
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN, LSTM

mongo_uri = "mongodb://localhost:27017"
client = mc(mongo_uri)
keti_db = client.keti_pattern_recognition

household_col = keti_db.household_info
weather_col = keti_db.weather_info

In [491]:
# TimeSlot In
hh_db_datas = household_col.find_one({"uid": "아파트1-104-1206"})
hh_db_datas

uid_in, timeslot = hh_db_datas['uid'], hh_db_datas['timeslot']

datelist = [
    dt.strptime(ts['time'], "%Y-%m-%d T%H:%M %z").date()
    for ts in timeslot
]
datelist = list(set(datelist))
datelist.sort()

ts_datas = {}
start_idx = 0
end_idx = 96
enl = 1

for date in datelist:
    ts_datas[date] = [ts['power'] *
                      enl for ts in timeslot[start_idx:end_idx]]
    start_idx = end_idx
    end_idx = end_idx + 96

ts_datas = pd.DataFrame(ts_datas).T
hh_datas = ts_datas.reset_index().copy()

hh_datas.rename(columns={"index": "date"}, inplace=True)
hh_datas['date'] = pd.to_datetime(hh_datas['date'])

hh_datas.set_index('date', inplace=True)

# Merging
merge_size = 4
merge_datas = pd.DataFrame()
for date in hh_datas.index:
    merge_ts = []
    new_ts_size = round(len(hh_datas.loc[date]) / merge_size)
    
    for idx in range(0,new_ts_size):
        merge_ts.append(
            hh_datas.loc[date][merge_size * idx:merge_size * (idx + 1)].sum()
        )
    merge_datas[date] = merge_ts
    
def get_season_no(month):
    if month in [3,4,5]:
        return 1 # 봄
    elif month in [6,7,8]:
        return 2 # 여름
    elif month in [9,10,11]:
        return 3 # 가을
    elif month in [12,1,2]:
        return 4 # 겨울
    
separate_datas_col = list(filter(lambda data: get_season_no(data.month) == 1, merge_datas.columns))
# print(separate_datas_col)
merge_datas = merge_datas[separate_datas_col]
merge_datas

,2018-05-01,2018-05-02,2018-05-03,2018-05-04,2018-05-05,2018-05-06,2018-05-07,2018-05-08,2018-05-09,2018-05-10,...,2019-04-21,2019-04-22,2019-04-23,2019-04-24,2019-04-25,2019-04-26,2019-04-27,2019-04-28,2019-04-29,2019-04-30
0,0.341,0.275,0.183,0.309,0.305,0.397,0.347,0.345,0.312,0.321,...,0.182,0.060,0.063,0.066,0.149,0.164,0.130,0.290,0.056,0.045
1,0.337,0.201,0.235,0.308,0.179,0.409,0.178,0.272,0.191,0.208,...,0.209,0.038,0.049,0.062,0.052,0.063,0.046,0.267,0.053,0.044
2,0.324,0.176,0.167,0.309,0.180,0.400,0.173,0.206,0.183,0.203,...,0.197,0.050,0.041,0.046,0.067,0.065,0.042,0.244,0.062,0.059
3,0.319,0.210,0.165,0.309,0.172,0.384,0.176,0.204,0.173,0.189,...,0.194,0.060,0.064,0.051,0.054,0.039,0.060,0.276,0.063,0.058
4,0.235,0.199,0.163,0.311,0.171,0.276,0.178,0.173,0.170,0.184,...,0.046,0.054,0.061,0.064,0.041,0.056,0.033,0.232,0.061,0.033
5,0.169,0.202,0.164,0.217,0.174,0.206,0.182,0.180,0.170,0.172,...,0.066,0.035,0.035,0.064,0.066,0.063,0.054,0.135,0.087,0.054
6,0.200,0.199,0.162,0.218,0.169,0.200,0.180,0.188,0.207,0.169,...,0.068,0.062,0.057,0.047,0.063,0.071,0.059,0.093,0.050,0.059
7,0.171,0.252,0.173,0.212,0.164,0.199,0.232,0.179,0.215,0.216,...,0.049,0.060,0.063,0.049,0.039,0.046,0.028,0.093,0.046,0.049
8,0.170,0.213,0.226,0.242,0.162,0.199,0.221,0.212,0.244,0.169,...,0.050,0.040,0.046,0.064,0.059,0.062,0.059,0.066,0.051,0.040
9,0.172,0.173,0.178,0.187,0.166,0.200,0.217,0.171,0.204,0.170,...,0.141,0.050,0.046,0.063,0.065,0.049,0.047,0.089,0.060,0.058


In [492]:
# Set Test Datas
# 계절별 10% 랜덤하게
def get_season_no(month):
    if month in [3,4,5]:
        return 1 # 봄
    elif month in [6,7,8]:
        return 2 # 여름
    elif month in [9,10,11]:
        return 3 # 가을
    elif month in [12,1,2]:
        return 4 # 겨울
    
test_merge_datas = pd.DataFrame();

for i in range(1,2):
    filter_list = list(filter(lambda date: get_season_no(date.month) == i, merge_datas.columns))
    test_list_idx = list()
    while True:
        filter_data = filter_list[ran.randrange(0,len(filter_list))]
        if filter_data not in test_list_idx:
            test_list_idx.append(filter_data)
            
        if len(test_list_idx) >= (len(filter_list) * 15 / 100):
            break;
    test_merge_datas = pd.concat([test_merge_datas, merge_datas[test_list_idx]], axis=1)
    merge_datas.drop(test_list_idx, axis=1, inplace=True)

test_merge_datas

,2018-05-01,2019-04-17,2019-04-06,2018-05-20,2019-03-15,2019-04-28,2019-03-23,2019-04-08,2018-05-26,2019-03-09,2019-04-02,2018-05-23,2018-05-24,2018-05-02
0,0.341,0.073,0.065,0.208,0.064,0.290,0.035,0.091,0.310,0.174,0.048,0.257,0.176,0.275
1,0.337,0.061,0.043,0.208,0.039,0.267,0.064,0.088,0.255,0.089,0.075,0.181,0.175,0.201
2,0.324,0.037,0.061,0.208,0.052,0.244,0.063,0.107,0.211,0.083,0.078,0.174,0.211,0.176
3,0.319,0.054,0.060,0.208,0.061,0.276,0.040,0.073,0.207,0.071,0.049,0.171,0.211,0.210
4,0.235,0.062,0.028,0.208,0.053,0.232,0.049,0.094,0.202,0.097,0.066,0.174,0.204,0.199
5,0.169,0.050,0.060,0.208,0.037,0.135,0.061,0.100,0.202,0.101,0.073,0.171,0.202,0.202
6,0.200,0.040,0.058,0.208,0.102,0.093,0.051,0.072,0.172,0.076,0.056,0.172,0.176,0.199
7,0.171,0.110,0.035,0.208,0.057,0.093,0.039,0.090,0.193,0.112,0.047,0.217,0.241,0.252
8,0.170,0.059,0.053,0.208,0.038,0.066,0.059,0.097,0.181,0.260,0.057,0.188,0.175,0.213
9,0.172,0.032,0.059,0.208,0.046,0.089,0.059,0.073,0.176,0.346,0.048,0.189,0.174,0.173


In [493]:
from functools import reduce

merge_datas
y = reduce(lambda acc, cur: cur + acc ,merge_datas.values.tolist(), [])
value_size = len(y)
print(value_size)

1872


In [494]:
# Config Training Datas
wt_db_datas = weather_col.find()
wt_datas = pd.DataFrame()
for wt in wt_db_datas:
    tmp = pd.DataFrame()
    tmp['date'] = [wt['date']]
    tmp['weather'] = [wt['weather']]
    tmp['avg_ta'] = round(float(wt['avgTa']))
    tmp['avg_rhm'] = round(float(wt['avgRhm']))
    
    wt_datas = pd.concat([wt_datas, tmp])

# weather 정수 인코딩
weather_count = Counter(wt_datas['weather'])
weather_integer = dict()
rank = 1
for key, count in weather_count.most_common():
    weather_integer[key] = rank
    rank += 1
pp.pprint(weather_integer)
wt_datas['weather_no'] = [weather_integer[weather] for weather in wt_datas['weather']]
wt_datas

def get_season_no(month):
    if month in [3,4,5]:
        return 1 # 봄
    elif month in [6,7,8]:
        return 2 # 여름
    elif month in [9,10,11]:
        return 3 # 가을
    elif month in [12,1,2]:
        return 4 # 겨울
    
# Date, Season Utils
wt_datas['season_no'] = [get_season_no(weather.month) for weather in wt_datas['date']] 
wt_datas['day_no'] = [weather.weekday() + 1 for weather in wt_datas['date']] 

sample_weather_col = ['season_no','day_no','weather_no','avg_ta', 'avg_rhm']
sample_weather_col_2 = ['date','season_no','day_no','weather','weather_no','avg_ta', 'avg_rhm']
wt_datas[sample_weather_col_2]

{'눈': 5,
 '박무': 4,
 '비': 3,
 '소나기': 7,
 '안개': 10,
 '안개비': 11,
 '연무': 2,
 '진눈깨비': 12,
 '채운': 8,
 '특이사항 없음': 1,
 '햇무리': 6,
 '황사': 9}


,date,season_no,day_no,weather,weather_no,avg_ta,avg_rhm
0,2018-05-01,1,2,연무,2,20,73
0,2018-05-02,1,3,비,3,15,90
0,2018-05-03,1,4,비,3,11,62
0,2018-05-04,1,5,특이사항 없음,1,14,46
0,2018-05-05,1,6,특이사항 없음,1,18,47
...,...,...,...,...,...,...,...
0,2019-04-26,1,5,비,3,8,82
0,2019-04-27,1,6,햇무리,6,12,53
0,2019-04-28,1,7,비,3,13,49
0,2019-04-29,1,1,햇무리,6,13,53


In [495]:
# Config Sample Datas - Padding
# Data Preprocessing
def get_samples(datas):
    samples = list()
    for col in datas:
        timeslot = datas[col].values.tolist()
        weather = wt_datas[wt_datas['date'] == col][sample_weather_col].values.tolist()[0]
        for time in range(1,25):
            samples.append(weather + timeslot[:time])

    print("Samples Before Padding Process")
    print(np.array(samples))
    print("\nTranining Sample Size : {}".format(len(samples)))
    
    return samples

def get_padding_samples(samples):    
    # Padding
    pad_samples = list()
    SAMPLE_MAX_LEN = max([len(s) for s in samples])
    print("Tranining Sample MAX_LEN : {}".format(SAMPLE_MAX_LEN))
    for sample in samples:
        err = SAMPLE_MAX_LEN - len(sample)
        if err == 0:
            pad_samples.append(sample)
        else:
            pad_data = [0 for i in range(0, err)]
            pad_samples.append(pad_data + sample)
    print("\nFinal Samples")
    print(np.array(pad_samples))
    
    return pad_samples
    
test_samples = get_samples(test_merge_datas)
test_samples = get_padding_samples(test_samples)

training_samples = get_samples(merge_datas)
training_samples = get_padding_samples(training_samples)

Samples Before Padding Process
[list([1, 2, 2, 20, 73, 0.341]) list([1, 2, 2, 20, 73, 0.341, 0.337])
 list([1, 2, 2, 20, 73, 0.341, 0.337, 0.324])
 list([1, 2, 2, 20, 73, 0.341, 0.337, 0.324, 0.31900000000000006])
 list([1, 2, 2, 20, 73, 0.341, 0.337, 0.324, 0.31900000000000006, 0.235])
 list([1, 2, 2, 20, 73, 0.341, 0.337, 0.324, 0.31900000000000006, 0.235, 0.169])
 list([1, 2, 2, 20, 73, 0.341, 0.337, 0.324, 0.31900000000000006, 0.235, 0.169, 0.2])
 list([1, 2, 2, 20, 73, 0.341, 0.337, 0.324, 0.31900000000000006, 0.235, 0.169, 0.2, 0.17099999999999999])
 list([1, 2, 2, 20, 73, 0.341, 0.337, 0.324, 0.31900000000000006, 0.235, 0.169, 0.2, 0.17099999999999999, 0.16999999999999998])
 list([1, 2, 2, 20, 73, 0.341, 0.337, 0.324, 0.31900000000000006, 0.235, 0.169, 0.2, 0.17099999999999999, 0.16999999999999998, 0.17200000000000001])
 list([1, 2, 2, 20, 73, 0.341, 0.337, 0.324, 0.31900000000000006, 0.235, 0.169, 0.2, 0.17099999999999999, 0.16999999999999998, 0.17200000000000001, 0.199])
 list

<ipython-input-495-f7267c0eac4d>:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  print(np.array(samples))


In [496]:
# Set Tranining Data
training_samples = np.array(training_samples)

train_X = training_samples[:,:-1]
train_y = training_samples[:,-1]

# Set Test Data
test_samples = np.array(test_samples)
test_X = test_samples[:,:-1]
test_y = test_samples[:,-1]

print("Input Data For Training")
print(np.array(train_X))
print()
print("Output Data For Training")
print(np.array(train_y))

print("Input Data For Test")
print(np.array(test_X))
print()
print("Output Data For Test")
print(np.array(test_y))

Input Data For Training
[[0.00e+00 0.00e+00 0.00e+00 ... 3.00e+00 1.10e+01 6.20e+01]
 [0.00e+00 0.00e+00 0.00e+00 ... 1.10e+01 6.20e+01 1.83e-01]
 [0.00e+00 0.00e+00 0.00e+00 ... 6.20e+01 1.83e-01 2.35e-01]
 ...
 [0.00e+00 0.00e+00 1.00e+00 ... 3.80e-02 6.10e-02 5.90e-02]
 [0.00e+00 1.00e+00 2.00e+00 ... 6.10e-02 5.90e-02 3.00e-02]
 [1.00e+00 2.00e+00 6.00e+00 ... 5.90e-02 3.00e-02 5.60e-02]]

Output Data For Training
[0.183 0.235 0.167 ... 0.03  0.056 0.058]
Input Data For Test
[[ 0.     0.     0.    ...  2.    20.    73.   ]
 [ 0.     0.     0.    ... 20.    73.     0.341]
 [ 0.     0.     0.    ... 73.     0.341  0.337]
 ...
 [ 0.     0.     1.    ...  0.218  0.2    0.206]
 [ 0.     1.     3.    ...  0.2    0.206  0.202]
 [ 1.     3.     3.    ...  0.206  0.202  0.201]]

Output Data For Test
[0.341 0.337 0.324 0.319 0.235 0.169 0.2   0.171 0.17  0.172 0.199 0.169
 0.17  0.215 0.212 0.257 0.203 0.198 0.197 0.196 0.211 0.335 0.306 0.305
 0.073 0.061 0.037 0.054 0.062 0.05  0.04  0.11 

In [331]:
model = Sequential()
# model.add(Embedding(value_size, input_length=28))
model.add(LSTM(128, input_shape=(28, 1),return_sequences=True, activation='softmax'))
model.add(Dense(1,activation='softmax'))
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

In [332]:
print(len(train_X), train_X.shape, train_X)
train_X = train_X.reshape(train_X.shape[0], train_X.shape[1], 1)
print(len(train_X), train_X.shape, train_X)
print(len(train_y), train_y.shape, train_y)

1872 (1872, 28) [[  0   0   0 ...   2  20  73]
 [  0   0   0 ...  20  73 342]
 [  0   0   0 ...  73 342 338]
 ...
 [  0   0   1 ...  39  62  60]
 [  0   1   2 ...  62  60  31]
 [  1   2   6 ...  60  31  57]]
1872 (1872, 28, 1) [[[  0]
  [  0]
  [  0]
  ...
  [  2]
  [ 20]
  [ 73]]

 [[  0]
  [  0]
  [  0]
  ...
  [ 20]
  [ 73]
  [342]]

 [[  0]
  [  0]
  [  0]
  ...
  [ 73]
  [342]
  [338]]

 ...

 [[  0]
  [  0]
  [  1]
  ...
  [ 39]
  [ 62]
  [ 60]]

 [[  0]
  [  1]
  [  2]
  ...
  [ 62]
  [ 60]
  [ 31]]

 [[  1]
  [  2]
  [  6]
  ...
  [ 60]
  [ 31]
  [ 57]]]
1872 (1872,) [342 338 325 ...  31  57  59]


In [333]:
model.fit(train_X, train_y, epochs=200, batch_size=1 ,verbose=2)

Epoch 1/200
1872/1872 - 7s - loss: 32817.1172 - accuracy: 0.0000e+00
Epoch 2/200
1872/1872 - 6s - loss: 32817.1211 - accuracy: 0.0000e+00
Epoch 3/200
1872/1872 - 6s - loss: 32817.1211 - accuracy: 0.0000e+00
Epoch 4/200
1872/1872 - 6s - loss: 32817.1211 - accuracy: 0.0000e+00
Epoch 5/200


KeyboardInterrupt: 

In [318]:
test = test_X[2]
test = test.reshape(28,1,1)
print(test)
result = model.predict(test, verbose=0)
print(result)

[[[  0]]

 [[  0]]

 [[  0]]

 [[  0]]

 [[  0]]

 [[  0]]

 [[  0]]

 [[  0]]

 [[  0]]

 [[  0]]

 [[  0]]

 [[  0]]

 [[  0]]

 [[  0]]

 [[  0]]

 [[  0]]

 [[  0]]

 [[  0]]

 [[  0]]

 [[  0]]

 [[  0]]

 [[  1]]

 [[  3]]

 [[  2]]

 [[  8]]

 [[ 55]]

 [[162]]

 [[160]]]
[[[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]]


In [385]:
print(train_X)
print(train_y)

one_hot_y = to_categorical(train_y,num_classes= value_size)
print(one_hot_y)

[[  0   0   0 ...   3  11  62]
 [  0   0   0 ...  11  62 184]
 [  0   0   0 ...  62 184 236]
 ...
 [  0   0   1 ...  39  62  60]
 [  0   1   2 ...  62  60  31]
 [  1   2   6 ...  60  31  57]]
[184 236 168 ...  31  57  59]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [497]:
model_2 = Sequential()
model_2.add(Embedding(value_size, 10, input_length=28))
model_2.add(LSTM(32))
model_2.add(Dense(value_size, activation='softmax'))
model_2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model_2.summary()

Model: "sequential_50"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 28, 10)            18720     
_________________________________________________________________
lstm_4 (LSTM)                (None, 32)                5504      
_________________________________________________________________
dense_40 (Dense)             (None, 1872)              61776     
Total params: 86,000
Trainable params: 86,000
Non-trainable params: 0
_________________________________________________________________


In [498]:
model_2.fit(train_X, one_hot_y, epochs=600, verbose=2)

Epoch 1/600
59/59 - 1s - loss: 7.1447 - accuracy: 0.0176
Epoch 2/600
59/59 - 0s - loss: 5.7161 - accuracy: 0.0321
Epoch 3/600
59/59 - 0s - loss: 5.4933 - accuracy: 0.0321
Epoch 4/600
59/59 - 0s - loss: 5.4561 - accuracy: 0.0321
Epoch 5/600
59/59 - 0s - loss: 5.4388 - accuracy: 0.0321
Epoch 6/600
59/59 - 0s - loss: 5.4287 - accuracy: 0.0321
Epoch 7/600
59/59 - 0s - loss: 5.4242 - accuracy: 0.0321
Epoch 8/600
59/59 - 0s - loss: 5.4203 - accuracy: 0.0321
Epoch 9/600
59/59 - 0s - loss: 5.4160 - accuracy: 0.0321
Epoch 10/600
59/59 - 0s - loss: 5.4151 - accuracy: 0.0321
Epoch 11/600
59/59 - 0s - loss: 5.4132 - accuracy: 0.0321
Epoch 12/600
59/59 - 0s - loss: 5.4127 - accuracy: 0.0321
Epoch 13/600
59/59 - 0s - loss: 5.4132 - accuracy: 0.0321
Epoch 14/600
59/59 - 0s - loss: 5.4115 - accuracy: 0.0321
Epoch 15/600
59/59 - 0s - loss: 5.4093 - accuracy: 0.0321
Epoch 16/600
59/59 - 0s - loss: 5.4104 - accuracy: 0.0321
Epoch 17/600
59/59 - 0s - loss: 5.4091 - accuracy: 0.0321
Epoch 18/600
59/59 - 0s

Epoch 142/600
59/59 - 0s - loss: 4.2126 - accuracy: 0.0919
Epoch 143/600
59/59 - 0s - loss: 4.1972 - accuracy: 0.0881
Epoch 144/600
59/59 - 0s - loss: 4.1779 - accuracy: 0.0913
Epoch 145/600
59/59 - 0s - loss: 4.1790 - accuracy: 0.0940
Epoch 146/600
59/59 - 0s - loss: 4.1602 - accuracy: 0.0962
Epoch 147/600
59/59 - 0s - loss: 4.1610 - accuracy: 0.0908
Epoch 148/600
59/59 - 0s - loss: 4.1630 - accuracy: 0.0908
Epoch 149/600
59/59 - 0s - loss: 4.1601 - accuracy: 0.0903
Epoch 150/600
59/59 - 0s - loss: 4.1587 - accuracy: 0.0908
Epoch 151/600
59/59 - 0s - loss: 4.1471 - accuracy: 0.0929
Epoch 152/600
59/59 - 0s - loss: 4.1283 - accuracy: 0.0962
Epoch 153/600
59/59 - 0s - loss: 4.1260 - accuracy: 0.0994
Epoch 154/600
59/59 - 0s - loss: 4.1354 - accuracy: 0.0967
Epoch 155/600
59/59 - 0s - loss: 4.1149 - accuracy: 0.0956
Epoch 156/600
59/59 - 0s - loss: 4.1152 - accuracy: 0.0988
Epoch 157/600
59/59 - 0s - loss: 4.1087 - accuracy: 0.0924
Epoch 158/600
59/59 - 0s - loss: 4.1021 - accuracy: 0.09

Epoch 281/600
59/59 - 0s - loss: 3.5362 - accuracy: 0.1640
Epoch 282/600
59/59 - 0s - loss: 3.5396 - accuracy: 0.1651
Epoch 283/600
59/59 - 0s - loss: 3.5213 - accuracy: 0.1725
Epoch 284/600
59/59 - 0s - loss: 3.5547 - accuracy: 0.1608
Epoch 285/600
59/59 - 0s - loss: 3.5062 - accuracy: 0.1683
Epoch 286/600
59/59 - 0s - loss: 3.4997 - accuracy: 0.1747
Epoch 287/600
59/59 - 0s - loss: 3.5043 - accuracy: 0.1704
Epoch 288/600
59/59 - 0s - loss: 3.4903 - accuracy: 0.1731
Epoch 289/600
59/59 - 0s - loss: 3.5582 - accuracy: 0.1608
Epoch 290/600
59/59 - 0s - loss: 3.5162 - accuracy: 0.1693
Epoch 291/600
59/59 - 0s - loss: 3.5078 - accuracy: 0.1736
Epoch 292/600
59/59 - 0s - loss: 3.4957 - accuracy: 0.1635
Epoch 293/600
59/59 - 0s - loss: 3.4845 - accuracy: 0.1715
Epoch 294/600
59/59 - 0s - loss: 3.4913 - accuracy: 0.1757
Epoch 295/600
59/59 - 0s - loss: 3.4816 - accuracy: 0.1731
Epoch 296/600
59/59 - 0s - loss: 3.4538 - accuracy: 0.1763
Epoch 297/600
59/59 - 0s - loss: 3.4599 - accuracy: 0.18

Epoch 420/600
59/59 - 0s - loss: 3.1298 - accuracy: 0.2196
Epoch 421/600
59/59 - 0s - loss: 3.1214 - accuracy: 0.2281
Epoch 422/600
59/59 - 0s - loss: 3.0614 - accuracy: 0.2340
Epoch 423/600
59/59 - 0s - loss: 3.0686 - accuracy: 0.2372
Epoch 424/600
59/59 - 0s - loss: 3.0373 - accuracy: 0.2425
Epoch 425/600
59/59 - 0s - loss: 3.0251 - accuracy: 0.2463
Epoch 426/600
59/59 - 0s - loss: 3.0228 - accuracy: 0.2425
Epoch 427/600
59/59 - 0s - loss: 3.0172 - accuracy: 0.2468
Epoch 428/600
59/59 - 0s - loss: 3.1039 - accuracy: 0.2238
Epoch 429/600
59/59 - 0s - loss: 3.0498 - accuracy: 0.2399
Epoch 430/600
59/59 - 0s - loss: 3.0361 - accuracy: 0.2404
Epoch 431/600
59/59 - 0s - loss: 3.0576 - accuracy: 0.2340
Epoch 432/600
59/59 - 0s - loss: 3.1837 - accuracy: 0.2067
Epoch 433/600
59/59 - 0s - loss: 3.1057 - accuracy: 0.2254
Epoch 434/600
59/59 - 0s - loss: 3.0438 - accuracy: 0.2340
Epoch 435/600
59/59 - 0s - loss: 3.0149 - accuracy: 0.2388
Epoch 436/600
59/59 - 0s - loss: 2.9996 - accuracy: 0.24

Epoch 559/600
59/59 - 0s - loss: 2.7388 - accuracy: 0.2991
Epoch 560/600
59/59 - 0s - loss: 2.7356 - accuracy: 0.3034
Epoch 561/600
59/59 - 0s - loss: 2.7290 - accuracy: 0.3018
Epoch 562/600
59/59 - 0s - loss: 2.7103 - accuracy: 0.3130
Epoch 563/600
59/59 - 0s - loss: 2.7078 - accuracy: 0.3141
Epoch 564/600
59/59 - 0s - loss: 2.7716 - accuracy: 0.2981
Epoch 565/600
59/59 - 0s - loss: 2.7562 - accuracy: 0.2981
Epoch 566/600
59/59 - 0s - loss: 2.7054 - accuracy: 0.3125
Epoch 567/600
59/59 - 0s - loss: 2.6985 - accuracy: 0.3173
Epoch 568/600
59/59 - 0s - loss: 2.7377 - accuracy: 0.2981
Epoch 569/600
59/59 - 0s - loss: 2.7281 - accuracy: 0.3072
Epoch 570/600
59/59 - 0s - loss: 2.7509 - accuracy: 0.3013
Epoch 571/600
59/59 - 0s - loss: 2.9834 - accuracy: 0.2634
Epoch 572/600
59/59 - 0s - loss: 3.2215 - accuracy: 0.2228
Epoch 573/600
59/59 - 0s - loss: 3.0273 - accuracy: 0.2521
Epoch 574/600
59/59 - 0s - loss: 2.9266 - accuracy: 0.2543
Epoch 575/600
59/59 - 0s - loss: 2.7606 - accuracy: 0.28

In [528]:
umm = 16
result = model_2.predict_classes([test_X[umm].tolist()])
print("result",y[result[0]])
print("real value", test_y[umm])

result 0.28800000000000003
real value 0.20299999999999999


In [487]:
train_y[4]

164

In [366]:
test_X[30]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        1,  6, 12,  3, 47, 36, 65, 64, 41, 50, 62])

In [ ]:
client.close()

In [400]:
test[2]

array([[0]])

0.22899999999999998

True